In [1]:
using Sobol
using Crystalline
using MPBUtils
using Crystalline: TEST_αβγs
using Crystalline: TEST_αβγ
using Brillouin
using Crystalline: dot
using Crystalline: norm
const INVERSION_PLANEGROUPS = Bool[0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1]
# Plane groups that have inversion (equivalently 2-fold rotational symmetry)
has_inversion(sgnum) = INVERSION_PLANEGROUPS[sgnum];

In [43]:
# For plane groups 10 and 16 we do not need to sample the DirectBasis vectors

sgnum = 16;
D = 2;
sg = sgnum;
epsout = 1.0
has_tr = true; res = 64; nbands = 40;
write_to_file = true;

In [44]:
function make_sobol_lattice(flat::Crystalline.AbstractFourierLattice{D}, modulation::AbstractVector{ComplexF64}, 
        cntr::Char, Gs::ReciprocalBasis{D}; expon::Real=1.25) where D
    
    flat = Crystalline.normscale(modulate(flat, modulation), expon, Gs)
    deleteat!(flat.orbits, 1), deleteat!(flat.orbitcoefs, 1) # trivial constant G=0 term
    map!(flat.orbitcoefs, flat.orbitcoefs) do coefs
        if has_inversion(sgnum)
            map!(x -> round(real(x), digits=5), coefs, coefs)            
        else
            map!(x -> round(x, digits=5), coefs, coefs)
        end
    end
    pflat = primitivize(flat, cntr)
    return pflat
end

make_sobol_lattice (generic function with 1 method)

In [45]:
cntr = centering(sg, D)
brs  = bandreps(sg, D, timereversal=has_tr)
lgs  = Crystalline.matching_littlegroups(brs, Val(D))
plgs = primitivize.(lgs, #=modw=# false)
kvs = brs.kvs
converted_kvs = [convert(Vector{Real}, kv(TEST_αβγs[2])) for kv in kvs]
flat = levelsetlattice(sgnum, 2, ntuple(_ -> 4, D));
num_coefs = length(getcoefs(flat))
N = 2num_coefs + 1; # Norm + phase + filling
s = skip(SobolSeq(N), 10000);

In [48]:
n = 10000 # 3n simulations per spacegroup
ids = 1:n
for id in ids
    (iszero(id%10) &&  println(id))
    flush(stdout)
    Rs = directbasis(sg, Val(D))
    Gs = reciprocalbasis(Rs)
    pRs = primitivize(Rs, cntr)
    
    next_sobol = next!(s)
    mod_r, mod_ϕ, filling = next_sobol[1:num_coefs], next_sobol[num_coefs+1:2*num_coefs], next_sobol[end]
    mod_ϕ *= 2π
    filling = 0.2 + 0.6*filling
    
    modulation = mod_r .* cis.(mod_ϕ) 

    pflat = make_sobol_lattice(flat, modulation, cntr, Gs)

    for (dielectric_enum, epsin) in enumerate((8, 12, 16))
        epsin != 16 && continue # only do epsilon=16
        for runtype in ["te", "tm"]
            filename = mpb_calcname(D, sg, id + (dielectric_enum-1)*n, res, runtype)
            filepath_symeig = joinpath("/home/gridsan/aligho/TopologyPaper", "sobol_symeigs/input/"*filename*".sh")
            write_to_file || continue
            open(filepath_symeig, "w") do io
                prepare_mpbcalc!(io, sg, pflat, pRs, filling, epsin, epsout, runtype;
                                    res=res, lgs=plgs, id = id + (dielectric_enum-1)*n, nbands=nbands)
            end
        end
    end
end

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [32]:
# Specific to plane group 2 (which is oblique)
sgnum = 2;
D = 2;
sg = sgnum;
epsout = 1.0
has_tr = true; res = 64; nbands = 40;
write_to_file = true;

In [33]:
cntr = centering(sg, D)
brs  = bandreps(sg, D, timereversal=has_tr)
lgs  = Crystalline.matching_littlegroups(brs, Val(D))
plgs = primitivize.(lgs, #=modw=# false)
kvs = brs.kvs
converted_kvs = [convert(Vector{Real}, kv(TEST_αβγs[2])) for kv in kvs]
flat = levelsetlattice(sgnum, 2, ntuple(_ -> 4, D));
num_coefs = length(getcoefs(flat))
N = 2num_coefs + 3; # Norm + phase + filling + b (second lattice vector) + γ (angle between lattice vectors)
s = skip(SobolSeq(N), 10000);

In [40]:
n = 10000 # 3n simulations per spacegroup
ids = 1:n
for id in ids
    (iszero(id%10) &&  println(id))
    flush(stdout)    
    next_sobol = next!(s)
    
    mod_r, mod_ϕ = next_sobol[1:num_coefs], next_sobol[num_coefs+1:2*num_coefs]
    filling = next_sobol[2*num_coefs+1]
    b = next_sobol[2*num_coefs+2]
    γ = next_sobol[2*num_coefs+3]
    
    b = 0.5 + (2-0.5)*b
    γ = π/6 + (2π/3)*γ
    Rs = crystal(1, b, γ)
    Gs = reciprocalbasis(Rs)
    pRs = primitivize(Rs, cntr)

    
    mod_ϕ *= 2π
    filling = 0.2 + 0.6*filling
    
    modulation = mod_r .* cis.(mod_ϕ) 

    pflat = make_sobol_lattice(flat, modulation, cntr, Gs)

    for (dielectric_enum, epsin) in enumerate((8, 12, 16))
        epsin != 16 && continue # only do epsilon=16
        for runtype in ["te", "tm"]
            filename = mpb_calcname(D, sg, id + (dielectric_enum-1)*n, res, runtype)
            filepath_symeig = joinpath("/home/gridsan/aligho/TopologyPaper", "sobol_symeigs/input/"*filename*".sh")
            write_to_file || continue
            open(filepath_symeig, "w") do io
                prepare_mpbcalc!(io, sg, pflat, pRs, filling, epsin, epsout, runtype;
                                    res=res, lgs=plgs, id = id + (dielectric_enum-1)*n, nbands=nbands)
            end
        end
    end
end

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222